In [1]:
!pip install pandas surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162990 sha256=c6d32228cf9df3dcdeac8c3f832984c94e6e1d24a949fd0d618534c8988c064b
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [38]:
import pandas as pd

file_path = "/content/shopsales23.xlsx"

data = pd.read_excel(file_path)

data = data.rename(columns={
    'name': 'order_id',
    'total': 'price',
    'billing_zip': 'zip_code',
    'billing_province': 'city',
    'lineitem_id': 'product_id'
})

print("Primeras filas del dataframe:")
print(data.head())

print("\nInformación del dataframe:")
print(data.info())

Primeras filas del dataframe:
  order_id financial_status             paid_at fulfillment_status  \
0  SHP5654             paid 2024-02-29 19:35:58          fulfilled   
1  SHP5653             paid 2024-02-29 17:52:03          fulfilled   
2  SHP5653              NaN                 NaT                NaN   
3  SHP5652             paid 2024-02-29 17:30:30          fulfilled   
4  SHP5652              NaN                 NaT                NaN   

         fulfilled_at accepts_marketing currency  subtotal  shipping    taxes  \
0 2024-03-04 15:44:28               yes      MXN    7990.0       0.0  1102.07   
1 2024-03-07 15:02:47               yes      MXN   12132.0       0.0  1673.38   
2                 NaT               NaN      NaN       NaN       NaN      NaN   
3 2024-03-04 15:39:38               yes      MXN   13280.0       0.0  1831.72   
4                 NaT               NaN      NaN       NaN       NaN      NaN   

   ...  phone receipt_number  duties billing_province_name  \


In [40]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

ratings_dict = {
    'userID': data['order_id'],
    'itemID': data['product_id'],
    'rating': data['price']
}
ratings_df = pd.DataFrame(ratings_dict)

reader = Reader(rating_scale=(0, ratings_df['rating'].max()))

data_surprise = Dataset.load_from_df(ratings_df[['userID', 'itemID', 'rating']], reader)

trainset, testset = train_test_split(data_surprise, test_size=0.25)

algo = SVD()

algo.fit(trainset)

predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: nan


nan

In [39]:
import pandas as pd

file_path = "/content/shopsales23.xlsx"
data = pd.read_excel(file_path)
data = data.rename(columns={
    'name': 'order_id',
    'total': 'price',
    'billing_zip': 'zip_code',
    'billing_province': 'city',
    'lineitem_id': 'product_id'
})

print("Primeras filas del dataframe:")
print(data.head())
print("\nInformación del dataframe:")
print(data.info())

def save_top_products_by_city(city, data, n=5, output_file="/content/top_products.xlsx"):
    city_data = data[data['city'] == city]
    if city_data.empty:
        print(f"No se encontraron datos para la ciudad {city}")
        return

    top_products = city_data['product_id'].value_counts().head(n)
    top_products_df = pd.DataFrame({
        'product_id': top_products.index,
        'sales_count': top_products.values
    })

    top_products_df.to_excel(output_file, index=False)
    print(f"Los {n} productos más vendidos en {city} se han guardado en {output_file}")

city_name = input("Ingrese el nombre de la ciudad: ")
save_top_products_by_city(city_name, data)

Primeras filas del dataframe:
  order_id financial_status             paid_at fulfillment_status  \
0  SHP5654             paid 2024-02-29 19:35:58          fulfilled   
1  SHP5653             paid 2024-02-29 17:52:03          fulfilled   
2  SHP5653              NaN                 NaT                NaN   
3  SHP5652             paid 2024-02-29 17:30:30          fulfilled   
4  SHP5652              NaN                 NaT                NaN   

         fulfilled_at accepts_marketing currency  subtotal  shipping    taxes  \
0 2024-03-04 15:44:28               yes      MXN    7990.0       0.0  1102.07   
1 2024-03-07 15:02:47               yes      MXN   12132.0       0.0  1673.38   
2                 NaT               NaN      NaN       NaN       NaN      NaN   
3 2024-03-04 15:39:38               yes      MXN   13280.0       0.0  1831.72   
4                 NaT               NaN      NaN       NaN       NaN      NaN   

   ...  phone receipt_number  duties billing_province_name  \


In [41]:
def get_city_from_order_id(order_id, data):
    city = data[data['order_id'] == order_id]['city'].iloc[0]
    return city

def save_top_products_by_city(city, data, n=5):
    city_data = data[data['city'] == city]

    if city_data.empty:
        print(f"No se encontraron datos para la ciudad {city}")
        return None

    top_products = city_data['product_id'].value_counts().head(n)
    top_product_names = top_products.index.tolist()

    return top_product_names

order_id = input("Ingrese su número de pedido: ")
city = get_city_from_order_id(order_id, data)

if city:
    top_products = save_top_products_by_city(city, data)

    if top_products:
        print(f"Los 5 productos más recomendados en {city} son:")
        for i, product in enumerate(top_products, 1):
            print(f"{i}. {product}")
    else:
        print("No se encontraron productos más vendidos para esta ciudad.")
else:
    print("Número de pedido no encontrado.")

Ingrese su número de pedido: SHP5641
Los 5 productos más recomendados en PUE son:
1. 0812028abb831e3fb4f10f4e65a521a5
2. d99da990a594b52b308774c56d8bd6f3
3. 7514d2ddfb90db25fe2f4a224ef0031d
4. 1f1a3c70e4d688dd675fee610e8ef9cc
5. e3940f10e0798800d5d69ba106420399


In [51]:
def save_top_products_by_city(data, n=10):
    city = input("Ingrese el nombre de la ciudad: ")

    city_data = data[data['city'] == city]

    if city_data.empty:
        print(f"No se encontraron datos para la ciudad {city}")
        return None
    top_products = city_data['product_id'].value_counts().head(n)

    top_product_names = top_products.index.tolist()

    return top_product_names

top_products = save_top_products_by_city(data)

if top_products:
    print("Los 10 productos más vendidos en la ciudad son:")
    for i, product in enumerate(top_products, 1):
        print(f"{i}. {product}")
else:
    print("No se encontraron productos más vendidos para esta ciudad.")

Ingrese el nombre de la ciudad: QRO
Los 10 productos más vendidos en la ciudad son:
1. e3940f10e0798800d5d69ba106420399
2. ad979ec8ced49ae72857226408549d27
3. e5e57a40bc13a59511ba0b2f8a185f29
4. c2f5bfd56350f528ccded7defc938faf
5. 803399174d8d638e584bf89436cc3dae
6. 16789c7e79991a9cbfced04ae9e2dd98
7. 391b9fbc0b7a3328d7b122c209999bf3
8. bb9d28b82d516da5cc948fd3abbe6a18
9. 464c48577b619c689ce0f5260887dead
10. 7c0b576016fb7639ff138f66afb50510


In [53]:
def top_months_by_sales(data, n=5):
    data['purchase_date'] = pd.to_datetime(data['purchase_date'], format='%d/%m/%Y %H:%M')

    data['month'] = data['purchase_date'].dt.month

    monthly_sales = data.groupby('month').size()

    top_months = monthly_sales.sort_values(ascending=False).head(n)

    return top_months
top_months = top_months_by_sales(data)

if not top_months.empty:
    print("Los 5 meses con más ventas son:")
    print(top_months)
else:
    print("No se encontraron datos de ventas.")

Los 5 meses con más ventas son:
month
11.0    365
6.0     291
5.0     228
2.0     208
3.0     199
dtype: int64


In [55]:
def top_months_by_sales(data, n=5):
    data['purchase_date'] = pd.to_datetime(data['purchase_date'], format='%d/%m/%Y %H:%M')

    data['month'] = data['purchase_date'].dt.month

    monthly_sales = data.groupby('month').size()

    top_months = monthly_sales.sort_values(ascending=False).head(n)

    return top_months.index

def recommend_next_purchase(order_id, data):
    order_data = data[data['order_id'] == order_id]

    if order_data.empty:
        print("No se encontraron datos para el número de pedido especificado.")
        return
    purchase_date = order_data['purchase_date'].iloc[0]

    top_months = top_months_by_sales(data)

    nearest_month = min(top_months, key=lambda x: abs(x - purchase_date.month))

    print(f"El mes más cercano RECOMENDADO para hacer la siguiente compra es: {nearest_month}")

order_id = input("Ingrese su número de pedido: ")

recommend_next_purchase(order_id, data)

Ingrese su número de pedido: SHP4875
El mes más cercano RECOMENDADO para hacer la siguiente compra es: 11.0


In [57]:
import pandas as pd
file_path = "/content/shopsales23.xlsx"
data = pd.read_excel(file_path)
data = data.rename(columns={
    'name': 'order_id',
    'total': 'price',
    'billing_zip': 'zip_code',
    'billing_province': 'city',
    'lineitem_id': 'product_id',
    'created_at': 'created_date',
    'fulfilled_at': 'fulfilled_date'
})

def top_fastest_shipping_months(data, n=5):
    data['created_date'] = pd.to_datetime(data['created_date'], format='%d/%m/%Y %H:%M')
    data['fulfilled_date'] = pd.to_datetime(data['fulfilled_date'], format='%d/%m/%Y %H:%M')

    data['shipping_duration'] = (data['fulfilled_date'] - data['created_date']).dt.days

    data['month'] = data['created_date'].dt.month
    avg_shipping_duration = data.groupby('month')['shipping_duration'].mean()

    top_fastest_shipping_months = avg_shipping_duration.sort_values().head(n)

    return top_fastest_shipping_months.index

fastest_shipping_months = top_fastest_shipping_months(data)

if not fastest_shipping_months.empty:
    print("Los 5 meses con envío más rápido son:")
    print(fastest_shipping_months)
else:
    print("No se encontraron datos de envío.")

Los 5 meses con envío más rápido son:
Index([7, 9, 5, 6, 4], dtype='int32', name='month')


In [58]:
def top_fastest_shipping_months(data, n=5):
    data['created_date'] = pd.to_datetime(data['created_date'], format='%d/%m/%Y %H:%M')
    data['fulfilled_date'] = pd.to_datetime(data['fulfilled_date'], format='%d/%m/%Y %H:%M')

    data['shipping_duration'] = (data['fulfilled_date'] - data['created_date']).dt.days

    data['month'] = data['created_date'].dt.month

    avg_shipping_duration = data.groupby('month')['shipping_duration'].mean()

    top_fastest_shipping_months = avg_shipping_duration.sort_values().head(n)

    return top_fastest_shipping_months.index

def recommend_next_purchase(order_id, data):
    order_data = data[data['order_id'] == order_id]

    if order_data.empty:
        print("No se encontraron datos para el número de pedido especificado.")
        return

    purchase_date = order_data['created_date'].iloc[0]

    fastest_shipping_months = top_fastest_shipping_months(data)

    nearest_month = min(fastest_shipping_months, key=lambda x: abs(x - purchase_date.month))

    print(f"El mes más cercano para hacer la siguiente compra para un envío más rápido es: {nearest_month}")

order_id = input("Ingrese su número de pedido: ")

recommend_next_purchase(order_id, data)

Ingrese su número de pedido: SHP4562
El mes más cercano para hacer la siguiente compra para un envío más rápido es: 9


In [66]:
def fraud_detection(order_id, data):
    order_data = data[data['order_id'] == order_id]

    if order_data.empty:
        print("No se encontraron datos para el número de pedido especificado.")
        return

    order_count = len(order_data)

    na_count = len(order_data[order_data['financial_status'] == 'NA'])

    fraud_ratio = na_count / order_count

    if fraud_ratio < 0.5:
        print("Este pedido tiene un alto riesgo de fraude.")
    else:
        print("Este pedido tiene un riesgo de fraude bajo.")

order_id = input("Ingrese su número de pedido: ")

fraud_detection(order_id, data)

Ingrese su número de pedido: SHP5653
Este pedido tiene un riesgo de fraude bajo.
